In [ ]:
from pyspark import SparkContext
import json
if __name__ == '__main__':
    
    
    read_review = sys.argv[1]
    read_business = sys.argv[2]
    write_file = sys.argv[3]
    flag = sys.argv[4]
    top_n = int(sys.argv[5])

    ans = dict()

    if flag == "spark":
        sc = SparkContext.getOrCreate()
        review_data = sc.textFile(read_review).map(lambda row: json.loads(row))
        bus_data = sc.textFile(read_business).map(lambda row: json.loads(row))
        rd = review_data.map(lambda kv: (kv['business_id'], kv['stars']))
        bd = bus_data.map(lambda kv: (kv['business_id'], kv['categories']))
        df1 = rd.filter(lambda kv: (kv[1] is not None)).map(lambda kv : (kv[0],float(kv[1]))).groupByKey().map(lambda kv : (kv[0], (sum(kv[1]),len(kv[1])) ))
        df2 = bd.filter(lambda kv: (kv[1] is not None) and (kv[1] is not "")).map(lambda kv: (kv[0],[value.strip() for value in kv[1].split(',')]))
        df3 = df2.join(df1)
        df4 = df3.map(lambda kv: kv[1]).flatMap(lambda kv: [(category, kv[1]) for category in kv[0]]).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
        df5 = df4.map(lambda kv: (kv[0],float(kv[1][0]/kv[1][1]))).takeOrdered(10, key=lambda kv: (-kv[1], kv[0]))
        ans['result'] = df5
    else:
        lines = open('review.json', encoding="utf8").readlines()
        rd = list(map(lambda line: {json.loads(line)['business_id'] : json.loads(line)['stars']}, lines))    
        d1 = dict()
        for item in rd:
            k = list(item.keys())[0]
            if k not in d1.keys():
                d1[k] = (float(item[k]), 1)
            else:
                new_value = d1.get(k)[0] + float(item[k])
                new_count = d1.get(k)[1] + 1
                d1.update({k: (new_value, new_count)})
        lines2 = open('business.json', encoding="utf8").readlines()
        bd = list(map(lambda l: { json.loads(l)['business_id'] : json.loads(l)['categories']}, lines2))
        d2 = dict()
        for item in bd:
            k = list(item.keys())[0]
            if (item[k] is not None) and (item[k] is not ""):
                d2[k] = [category.strip() for category in item[k].split(',')]
        
        
        d1_d2 = dict()
        for id, score_count in d1.items():
            if None is not d2.get(id):
                for category in d2.get(id):
                    if category not in d1_d2.keys():
                        d1_d2[category] = score_count
            else:
                new_value = d1_d2.get(category)[0] + score_count[0]
                new_count = d1_d2.get(category)[1] + score_count[1]
                d1_d2.update({category: (new_value, new_count)})
        
        ans1 = {k: float(v[0] / v[1]) for k, v in d1_d2.items()}

        ans['result'] = sorted(ans1.items(), key=lambda kv: (-kv[1], kv[0]))[:int(10)]
    json.dump(ans, open(write_file,"w"))   

    